In [1]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [213]:
# define functions to compute percentile by hand
# dataOrderer:         compute rank and order original data acccordint to it
# compute_p_threshold: compute the data calue associated to the desired 
#                      percentile p
# compute_percentiles: cycle on the selected percentiles range 
#                      and return percentiles


def dataOrderer(data):
    """
    Compute the data rank of input data. 
    Return the ordered input data. 
    
    data: input data. 
          Takes any data type compatible with "scipy.stats.rankdata" 
    
    return: ordered data. Same tadatype and size of data.
    
    """ 
    N = len(data)
    
    rank = stats.rankdata(data, method='ordinal')

    # order data according to rank
    ordered_data = np.zeros(N)
    for d, r in zip(data, rank):
        ordered_data[int(r) - 1 ] = d
    
    return ordered_data


def compute_p_threshold(data, p, decimal=3):
    
    """
    Compute the threshold value (score) associated to 
    the "p-th" percentileof data. 
    
    data: input data. Takes any data type compatible 
          with "scipy.stats.rankdata" 
    
    decimal: decimal digits used to round the output. 
             Default 'digit = 3'
    
    return: score value associated to p % of data. 
            The returned value is rounded to "decimal" significant digits.
    
    """
    
    N = len(data)
    ordered_data = dataOrderer(data)
        
    x = p*(N + 1)
    k = int(x)
    d = x % 1
    last_k = -1
    
    p_thresh = None
    
    if (k > 0) & (k <= N - 1):        
        
        if k == last_k:
            p_thresh = ordered_data[k-1] + \
                        d*(ordered_data[k]-ordered_data[k-1])
            
        else:
            p_thresh = ordered_data[k-1] + \
                        d*(ordered_data[k]-ordered_data[k-1])
            last_k = k
            
    elif k == 0:            
        p_thresh = ordered_data[0]             
        
    elif k > N-1:            
        p_thresh = ordered_data[N-1] 
        
    return round(p_thresh, decimal)

    
def compute_percentiles(data, decimal=8, percentiles=np.arange(0,101,1)):
    """
    Compute percintiles of input data.
    
    ordered_data: input data. Ordered data by 'ordinal' rank function.
                  (see method='ordinal' of scipy.stats.rankdata)
    
    decimal: decimal digits used to round the output. Default 'digit = 3'
    
    percentiles: iterable. Percentage values at which compute 
                 the percentage. 
                 Default 'percentiles=np.arange(0, 1, 101)'
                 compute "standard" percentiles from 0 to 100 % 
                 with step of 1%.
                 
    return: computed percentiles
    
    """
    
    percentiles = percentiles / 100
    
    computed_percentiles = {}
    
    for p in percentiles:      
        computed_percentiles[p] = compute_p_threshold(data, p, decimal)
        
    return computed_percentiles

In [214]:
# test
# simulate data
N = 10000
data = np.random.randn(N)

# define percentile to use
percentiles = np.arange(0,101,1)

# numpy built-in function
numpy_method   = np.percentile(data, percentiles, method='weibull')
# my function
my_pers = [x for x in compute_percentiles(data, percentiles=percentiles).values()]
by_hand_method = np.array(my_pers)

# compute and print differnce till 8-th digit
diff = numpy_method - by_hand_method
print('diff\n', np.round(diff, 8))

diff
 [ 0.  0. -0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0. -0.  0. -0.  0.  0.
  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0.  0.  0. -0. -0. -0.
  0.  0.  0.  0.  0. -0.  0. -0. -0.  0. -0. -0.  0.  0. -0.  0.  0. -0.
  0.  0.  0.  0.  0. -0.  0.  0. -0. -0.  0. -0. -0.  0. -0.  0. -0. -0.
 -0. -0. -0.  0.  0. -0. -0.  0. -0. -0.  0. -0.  0.  0. -0. -0.  0.  0.
  0.  0.  0. -0. -0.  0. -0.  0. -0.  0. -0.]


In [229]:
 resistivity = [95.177,
                95.156,
                95.193,
                95.195,
                95.144,
                95.061,
                95.159,
                95.119,
                95.106,
                95.092,
                95.199,
                95.168]

print("p[%]\t\tValue")
print(23*"-")
for key, value in compute_percentiles(resistivity, percentiles=np.arange(0,101,10)).items():
    print("{}%\t-->\t{}".format((key*100),value ))

p[%]		Value
-----------------------
0.0%	-->	95.061
10.0%	-->	95.0703
20.0%	-->	95.1004
30.0%	-->	95.1177
40.0%	-->	95.1464
50.0%	-->	95.1575
60.0%	-->	95.1662
70.0%	-->	95.1786
80.0%	-->	95.1938
90.0%	-->	95.1978
100.0%	-->	95.199
